# Tests

In [1]:
import os
import sys
import logging
import pandas as pd

#dir = os.path.abspath(__file__)
#dir    = os.getcwd()
#pkgdir = os.path.dirname(os.path.dirname(dir)

sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))
from lib.nanuq import Nanuq
from lib.gapp import Phenotips


In [5]:
os.chdir("D:\\HSJ\\tmp")
os.listdir()

['emg', 'foo.csv', 'A00516_0445']

In [15]:
file    = "A00516_0445\\emg_batch_manifest.csv"
def get_samples_list(file):
    samples = []
    with open(file, 'r') as fh:
        for line in fh.readlines():
            cols = line.split(',')
            samples.append(cols[5])
    samples.pop(0)
    return samples
print(get_samples_list(file))


['GM231651', 'GM231626', 'GM231624', 'GM231632', 'GM231627', 'GM231615', 'GM231667', 'GM231668', '23-05982-T1', '23-06384-T1', '23-06383-T1', '3042652455', '3042642360', '3042645886', '3052768938', '3052768961', '3052768951', '3052768942']
